# Interpretable AI: The Future of Patient-Centric Healthcare with GraphRAGThis notebook demonstrates key concepts and implementations related to Graph Retrieval-Augmented Generation (GraphRAG) in healthcare applications, with a focus on interpretability and patient-centric care.

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
from rdflib import Graph, URIRef, Literal, Namespace
import shap
import warnings
warnings.filterwarnings('ignore')

## 1. Building a Knowledge GraphLet's create a simple medical knowledge graph using RDFLib to demonstrate how GraphRAG structures healthcare data.

In [ ]:
# Create a medical knowledge graph
g = Graph()
MED = Namespace("http://healthcare.org/")

# Add medical relationships
g.add((URIRef(MED.Diabetes), URIRef(MED.hasSymptom), Literal("Increased thirst")))
g.add((URIRef(MED.Diabetes), URIRef(MED.hasSymptom), Literal("Frequent urination")))
g.add((URIRef(MED.Diabetes), URIRef(MED.hasTreatment), Literal("Insulin therapy")))

# Print the graph
print("Medical Knowledge Graph Relations:")
for s, p, o in g:
    print(f"{s.n3()} {p.n3()} {o.n3()}")

## 2. Implementing SHAP for Model InterpretabilityWe'll demonstrate how to use SHAP values to explain model predictions in a healthcare context.

In [ ]:
# Create sample healthcare dataset
np.random.seed(42)
n_samples = 1000

data = pd.DataFrame({
    'age': np.random.normal(65, 15, n_samples),
    'blood_pressure': np.random.normal(130, 20, n_samples),
    'glucose': np.random.normal(100, 25, n_samples),
    'bmi': np.random.normal(28, 5, n_samples)
})

# Create target variable (diabetes risk)
data['diabetes_risk'] = (data['age'] * 0.3 + 
                        data['blood_pressure'] * 0.2 + 
                        data['glucose'] * 0.4 + 
                        data['bmi'] * 0.1 + 
                        np.random.normal(0, 10, n_samples))
data['diabetes_risk'] = (data['diabetes_risk'] > data['diabetes_risk'].mean()).astype(int)

# Train a simple model
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

X = data.drop('diabetes_risk', axis=1)
y = data['diabetes_risk']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Calculate SHAP values
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_test)

# Plot SHAP summary
shap.summary_plot(shap_values, X_test)

## 3. Error Handling and Best PracticesDemonstrate proper error handling in healthcare AI applications.

In [ ]:
def validate_patient_data(patient_data):
    """Validate patient data with proper error handling"""
    try:
        required_fields = ['age', 'blood_pressure', 'glucose', 'bmi']
        
        # Check for missing fields
        missing_fields = [field for field in required_fields 
                         if field not in patient_data]
        if missing_fields:
            raise ValueError(f"Missing required fields: {missing_fields}")
            
        # Validate value ranges
        if not (0 <= patient_data['age'] <= 120):
            raise ValueError("Age must be between 0 and 120")
            
        if not (60 <= patient_data['blood_pressure'] <= 200):
            raise ValueError("Blood pressure must be between 60 and 200")
            
        return True
        
    except Exception as e:
        print(f"Error validating patient data: {str(e)}")
        return False

# Test the validation
test_data = {
    'age': 25,
    'blood_pressure': 120,
    'glucose': 95,
    'bmi': 22
}

is_valid = validate_patient_data(test_data)
print(f"Data validation result: {is_valid}")

## ConclusionThis notebook has demonstrated key concepts in implementing interpretable AI for healthcare using GraphRAG, including:- Building knowledge graphs for healthcare data- Using SHAP for model interpretability- Implementing proper error handlingThese implementations help ensure transparent, reliable, and ethical AI systems in healthcare applications.